# 张量
张量是一种特殊的数据结构，与数组和矩阵非常相似。在 PyTorch 中，我们使用张量对模型的输入和输出以及模型的参数进行编码。

张量类似于 NumPy 的 ndarrays，不同之处在于张量可以在 GPU 或其他硬件加速器上运行。事实上，张量和 NumPy 数组通常可以共享相同的底层内存，从而无需复制数据（请参阅 [Bridge with NumPy](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label)）。张量也针对自动微分进行了优化（我们将在稍后的 [Autograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) 部分中看到更多相关信息）。如果您熟悉 ndarrays，那么您将对 Tensor API 不陌生。如果不熟悉，那么就来学习吧！

In [23]:
import torch
import numpy as np

## 初始化一个张量
张量可以通过多种方式初始化。下面来看看几个例子：

**直接从原始数据初始化**
张量可以直接从数据中创建。数据类型是自动推断的。

In [24]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)

tensor([[1, 2],
        [3, 4]])


**从numpy数组初始化**
可以从 NumPy 数组创建张量（反之亦然 - 请参阅 [Bridge with NumPy](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label)）。

In [25]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


**从其它张量初始化**
除非明确覆盖，否则新张量保留参数张量的属性（形状、数据类型）。

In [26]:
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.6087, 0.3386],
        [0.8160, 0.8963]]) 



**具有随机值或者常量值**
下面的shape是一个指定了张量的元组。在下面的函数中，它决定了输出张量的维度。

In [27]:
shape = (2, 3, )
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.5121, 0.6116, 0.9648],
        [0.1209, 0.2535, 0.4767]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## 张量属性
张量属性描述了张量的维度、数据类型和存储设备

In [28]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## 张量的操作
[这里](https://pytorch.org/docs/stable/torch.html)全面描述了100多个张量操作，包括算术，线性代数，矩阵操作（转置，索引，切片），采样等。

每一个操作都可以在GPU上运行（通常比GPU更快）。如果您使用的是COLAB，请通过 运行时>更改运行时类型>GPU 操作链路分配GPU。

默认情况下都在CPU上创建张量。我们需要使用.to方法将张量显式移动到GPU（在检查GPU可用性之后）。请记住，在时间和内存方面，跨设备复制大量张量会非常损耗性能！

In [29]:
if torch.cuda.is_available():
    print("cuda available")
    tensor = tensor.to("cuda")

cuda available


从列表中尝试一些操作。如果您熟悉Numpy API，则会发现使用张量API轻而易举。

**标准的numpy型索引和切片**

In [30]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:, 1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**拼接张量**
您可以使用 torch.cat 沿给定维度连接一系列张量。另见 [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)，另外一个张量拼接方法，但与 torch.cat 略有不同。

In [31]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


**算数运算**

In [32]:
# 这计算两个张量之间的矩阵乘法。 y1、y2、y3 将具有相同的值
# ``tensor.T`` 返回张量的转置
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)
y3 = torch.rand_like(y1)
print(f"y1:{y1}\ny2:{y2}\ny3:{y3}")
torch.matmul(tensor, tensor.T, out=y3)

# 这将计算元素乘积。 z1、z2、z3 将具有相同的值
z1 = tensor * tensor
z2 = tensor.mul(tensor)
z3 = torch.rand_like(tensor)
print(f"z1:{z1}\nz2:{z2}\nz3:{z3}")
torch.mul(tensor, tensor, out=z3)

y1:tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
y2:tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
y3:tensor([[5.4285e-01, 4.2646e-01, 1.8945e-01, 2.9056e-01],
        [3.9075e-01, 1.7919e-01, 2.8425e-01, 2.0635e-01],
        [7.1765e-02, 6.7266e-01, 6.1798e-04, 2.1421e-01],
        [1.6978e-01, 2.4059e-01, 1.8514e-01, 7.5396e-01]])
z1:tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
z2:tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
z3:tensor([[0.6220, 0.2687, 0.6980, 0.9420],
        [0.2777, 0.9354, 0.7016, 0.1426],
        [0.9863, 0.8366, 0.6872, 0.7848],
        [0.3596, 0.7253, 0.9320, 0.6268]])


tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

**单元素张量**
如果你有一个单元素张量，例如通过将一个张量的所有值相加成一个值，你可以使用 item() 将它转换为 Python 数值。

In [33]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


**In-place operations**（原地操作，就是在原张量上操作）
将结果存储到操作数中的操作称为就地操作。它们由 _ 后缀表示。例如：x.copy_(y),x.t_()，会改变x。

In [34]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


> note
> 就地操作可以节省一些内存，但在计算导数时可能会出现问题，因为会立即丢失原张量信息。因此，不鼓励使用它们。

## 桥接 Numpy
CPU 上的张量和 NumPy 数组会共享它们的底层内存，改变一个就会改变另一个（可以理解为引用变量，张量和 Numpy 数组都是引用地址）。

### 张量转化 Numpy 数组

In [35]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


张量的变化将会引起 Numpy 数组的变化

In [36]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### Numpy 数组转化张量

In [37]:
n = np.ones(5)
t = torch.from_numpy(n)

Numpy 数组的变化将会引起张量的变化

In [38]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
